In [1]:
import pandas as pd
import seaborn as sns

In [3]:
rdata = pd.read_csv('../data/syrian-refugee-counts.csv')

as1 = rdata.Refugees != '*'
as2 = rdata.AsylumSeekers != '*'
rdata = rdata[as1 & as2]

rdata['Refugees'] = rdata['Refugees'].astype(float)
rdata['AsylumSeekers'] = rdata['AsylumSeekers'].astype(float)

# pad with 0s
rdata = rdata.fillna(0)
rdata['T'] = rdata['Refugees'] + rdata['AsylumSeekers']

# only use subset
rdata = rdata[['Destination', 'T', 'Year']]

In [4]:
d = set(rdata.Destination)
y = set(rdata.Year)
p = pd.MultiIndex.from_product([d,y], names=['destinations', 'year'])
rdata.reindex(p)
rdata.head(5)

,Destination,T,Year
0,Lebanon,1200,1968
1,Lebanon,1200,1969
2,Lebanon,500,1979
3,Italy,2,1980
4,Lebanon,500,1980


In [7]:
pop = pd.read_csv('../data/demographics-wb.csv')
pop = pop[['Country Name',  '2011', '2012', '2013','2014']]
pop.columns = ['Destination', '2011_pop', '2012_pop', '2013_pop', '2014_pop']

In [8]:
# Find top10
r = rdata
r = r.drop(['Year'], 1)
g = r.groupby('Destination').sum()
top10 = g.sort('T', ascending=False).head(10)
top10

,T
Destination,
Turkey,2393091
Lebanon,2134927
Jordan,829210
Iraq,528157
Egypt,283099
Germany,256993
Sweden,137136
Netherlands,51502
Libya,39428


In [19]:
rd = rdata[rdata['Year'] > 2010]
d_pivot = pd.pivot_table(rd, columns=['Destination', 'Year'], values=['T'])
df = d_pivot.unstack()
# to output:
df.reset_index().to_json(orient='records', path_or_buf='../app/src/data/stats.json')

In [20]:
rfs = df.reset_index()
rfs = rfs.merge(pop, on='Destination', how='left')
rfs['rate_avg'] = rfs[2014] / rfs['2014_pop']
rfs = rfs.sort(2014, ascending=0)
rfs.columns = ['level_0', 'Destination', 'ref_2011', 'ref_2012', 'ref_2013', 'ref_2014', 'pop_2011',
       'pop_2012', 'pop_2013', 'pop_2014', 'rate_avg']
rfs.head()


,level_0,Destination,ref_2011,ref_2012,ref_2013,ref_2014,pop_2011,pop_2012,pop_2013,pop_2014,rate_avg
122,T,Turkey,224.0,248666,585711,1558149.0,73058638,73997128,74932641,75837020,0.020546
67,T,Lebanon,245.5,127181,851615,573955.5,4382790,4424888,4467390,4510301,0.127254
55,T,Iraq,1714.0,64771,213973,235421.0,31760020,32578209,33417476,34278364,0.006868
41,T,Egypt,94.0,12836,131659,138381.0,79392466,80721874,82056378,83386739,0.001660
47,T,Germany,14868.0,24151,31819,70585.0,81797673,80425823,80645605,80889505,0.000873


In [21]:
rfs.sort('ref_2014', ascending=0).to_json(orient='records', path_or_buf='../app/src/data/stats.json')

In [28]:
rfs[:4]['ref_2014'].sum() / rfs['ref_2014'].sum()

0.90692555423235355

In [31]:
 rfs[4:]['ref_2014'].sum()

257172.0

In [32]:
257172.0 / 50000

5.14344